In [ ]:
from bloqade.gemini import logical as gemini_logical
from tsim import Circuit
from bloqade import qubit, squin
from bloqade.lanes.logical_mvp import (
    compile_to_physical_stim_program,
    kernel,
    set_detector,
    set_observable,
    Decoder,
)

In [ ]:
@kernel
def main():
    # see arXiv: 2412.15165v1, Figure 3a
    reg = qubit.qalloc(5)
    squin.broadcast.t(reg)

    squin.broadcast.sqrt_x([reg[0], reg[1], reg[4]])
    squin.broadcast.cz([reg[0], reg[2]], [reg[1], reg[3]])
    squin.broadcast.sqrt_y([reg[0], reg[3]])
    squin.broadcast.cz([reg[0], reg[3]], [reg[2], reg[4]])
    squin.sqrt_x_adj(reg[0])
    squin.broadcast.cz([reg[0], reg[1]], [reg[4], reg[3]])
    squin.broadcast.sqrt_y_adj(reg)

    measurements = gemini_logical.terminal_measure(reg)

    for i in range(len(reg)):
        set_detector(measurements[i])
        set_observable(measurements[i], i)

In [ ]:
program = compile_to_physical_stim_program(main)

In [ ]:
program.diagram(height=1000)

In [ ]:
def simulate_program(program: Circuit, shots: int):
    sampler = program.compile_detector_sampler()
    return sampler.sample(shots, separate_observables=True)

In [ ]:
detector_bits, logical_bits = simulate_program(program, shots=100_000)

In [ ]:
detector_bits

In [ ]:
logical_bits

In [ ]:
dem = program.detector_error_model(approximate_disjoint_errors=True)
decoder = Decoder(dem)

In [ ]:
corrected_logical_bits = decoder.decode(detector_bits[:1000]) ^ logical_bits[:1000]
corrected_logical_bits